In [ ]:
import pandas as pd
import numpy as np

def eyring_equation(temperature, delta_H_dagger, delta_S_dagger):
    if temperature == 0 or pd.isna(temperature):
        return np.nan 
    
    k_B = 1.380649e-23  # Boltzmann constant in J/K
    h = 6.62607015e-34  # Planck's constant in J·s
    R = 8.314  # Gas constant in J/(mol·K)
    temperature_kelvin = temperature + 273.15
    rate_constant = (k_B * temperature_kelvin / h) * np.exp(-delta_H_dagger / (R * temperature_kelvin)) * np.exp(delta_S_dagger / R)
    return rate_constant

def calculate_delta_delta_G(temperature, ee, delta_H_dagger_major, delta_S_dagger_major, delta_H_dagger_minor, delta_S_dagger_minor):
    if temperature == 0 or pd.isna(temperature):
        return np.nan 
    
    k_major = eyring_equation(temperature, delta_H_dagger_major, delta_S_dagger_major)
    k_minor = eyring_equation(temperature, delta_H_dagger_minor, delta_S_dagger_minor)
    
    ee_factor = (1 + ee) / (1 - ee)
    delta_delta_G = 8.314 * (temperature + 273.15) * np.log(ee_factor)
    delta_delta_G /= 4184  
    
    return delta_delta_G

# Load the data
file_path = 'reordered_final_merged_cleaned_qqm_molecule_charges.xlsx'
df = pd.read_excel(file_path)

# Filter out rows where 'ee' is equal to 1
df = df[(df['Temperature/C'] > 0) & (df['ee'] != 1)].dropna(subset=['Temperature/C'])

# Constants for delta H and delta S values
constant_delta_H_dagger_major = 50000  # in J/mol
constant_delta_S_dagger_major = -100   # in J/(mol·K)
constant_delta_H_dagger_minor = 51000  # in J/mol
constant_delta_S_dagger_minor = -110   # in J/(mol·K)

# Calculate Scaled ΔΔG (kcal/mol)
df['Scaled ΔΔG (kcal/mol)'] = df.apply(
    lambda row: calculate_delta_delta_G(
        temperature=row['Temperature/C'],
        ee=row['ee'],  # Observed %ee
        delta_H_dagger_major=constant_delta_H_dagger_major,
        delta_S_dagger_major=constant_delta_S_dagger_major,
        delta_H_dagger_minor=constant_delta_H_dagger_minor,
        delta_S_dagger_minor=constant_delta_S_dagger_minor
    ), axis=1
)

# Display and save the output
print(df[['Temperature/C', 'ddG', 'ee', 'Scaled ΔΔG (kcal/mol)']].head())
df.to_excel('scaled_output_ddG_constant_ee_kcal.xlsx', index=False)
